In [ ]:
import numpy as np
import math

# ------------------------
# Helpers
# ------------------------

def random_on_sphere(dim):
    vec = np.random.normal(0, 1, dim)
    return vec / np.linalg.norm(vec)

def sample_vMF(mu, kappa, norm):
    dim = len(mu)
    if kappa < 1e-6:
        return norm * random_on_sphere(dim)
    noise = np.random.normal(0, 1 / np.sqrt(kappa), size=dim)
    sample = mu + noise
    return norm * sample / np.linalg.norm(sample)

# ------------------------
# Gibbs Sampling
# ------------------------

def run_gibbs(beta_human, beta_ai, n_people=20, n_f_h=20, n_f_ai=20,
              n_steps=2000, burn_in=1000, thinning=10):
    J_human = np.random.normal(0, 1, size=(n_people, n_f_h))
    J_ai = np.random.normal(0, 1, size=(n_people, n_f_ai))

    X = np.sqrt(n_people) * random_on_sphere(n_people)
    F_human = np.sqrt(n_f_h) * random_on_sphere(n_f_h)
    F_ai = np.sqrt(n_f_ai) * random_on_sphere(n_f_ai)

    overlaps = []

    for step in range(n_steps):
        h_X = (beta_human / np.sqrt(n_people + n_f_h)) * J_human @ F_human + \
              (beta_ai / np.sqrt(n_people + n_f_ai)) * J_ai @ F_ai
        X = sample_vMF(h_X / np.linalg.norm(h_X + 1e-9), np.linalg.norm(h_X), np.sqrt(n_people))

        h_F_h = (beta_human / np.sqrt(n_people + n_f_h)) * J_human.T @ X
        F_human = sample_vMF(h_F_h / np.linalg.norm(h_F_h + 1e-9), np.linalg.norm(h_F_h), np.sqrt(n_f_h))

        h_F_ai = (beta_ai / np.sqrt(n_people + n_f_ai)) * J_ai.T @ X
        F_ai = sample_vMF(h_F_ai / np.linalg.norm(h_F_ai + 1e-9), np.linalg.norm(h_F_ai), np.sqrt(n_f_ai))

        if step >= burn_in and (step - burn_in) % thinning == 0:
            overlap = np.dot(F_human, F_ai) / (np.linalg.norm(F_human) * np.linalg.norm(F_ai))
            overlaps.append(overlap)

    return np.array(overlaps)

# ------------------------
# Simulation and Save Results
# ------------------------

beta_values = np.array([0,0.5,1,1.25,1.5,2.00])
results = {}

for beta_h in beta_values:
    for beta_a in beta_values:
        key = (round(beta_h, 2), round(beta_a, 2))
        overlaps = run_gibbs(beta_h, beta_a)
        results[key] = overlaps

# Save the results to a file
np.save('gibbs_results.npy', results)
